# _Skin Cancer - Classification Project_

_Bu projede deri kanserini algılayan bir **CNN modeli** geliştireceğiz. Modeli kaydedip Streamlit uygulamasıa çevireceğiz ve Hugginface de çalışır hale getireceğiz._

### _İmport_

In [1]:
# OpenCV kütüphanesini import ediyoruz, görüntü işleme ve video/frame işlemleri için kullanılır
import cv2

# Pandas kütüphanesini import ediyoruz, veri analizi ve tablo şeklinde veri yönetimi için kullanılır
import pandas as pd

# NumPy kütüphanesini import ediyoruz, sayısal hesaplamalar ve matris işlemleri için kullanılır
import numpy as np

# scikit-learn kütüphanesinden train_test_split fonksiyonunu import ediyoruz,
# veriyi eğitim ve test setlerine ayırmak için kullanılır
from sklearn.model_selection import train_test_split

# Keras kütüphanesinden gerekli modülleri import ediyoruz
# Sequential: katman katman model oluşturmak için
# Conv2D: 2D evrişim katmanı (görüntülerde özellik çıkarımı için)
# Dense: tam bağlantılı katman (sınıflandırma veya regresyon için)
# Flatten: çok boyutlu veriyi tek boyuta indirger
# Input: modelin giriş katmanı
# MaxPooling2D: evrişim katmanından sonra boyut küçültmek için max pooling uygular
# Dropout: aşırı öğrenmeyi (overfitting) önlemek için rastgele nöronları kapatır
# BatchNormalization: modelin daha hızlı ve stabil öğrenmesini sağlar
# Reshape: verinin şeklini değiştirmek için
from keras.models import Sequential
from keras.layers import Conv2D, Dense, Flatten, Input, MaxPooling2D, Dropout, BatchNormalization, Reshape

# İşletim sistemi ile ilgili işlemler yapmak için os modülünü import ediyoruz
# Örneğin dosya/dizin kontrolleri, dosya yolları oluşturma gibi
import os

In [2]:
# Python'da uyarı mesajlarını yönetmek için warnings modülünü import ediyoruz
import warnings

# Tüm uyarı mesajlarını görmezden gelmek için filterwarnings ile 'ignore' ayarını yapıyoruz
# Bu sayede ekranda gereksiz uyarılar görünmez ve kod çıktısı daha temiz olur
warnings.filterwarnings('ignore')

### _Eda_

In [3]:
# Sınıf etiketlerini bir liste halinde tanımlıyoruz
# 'Cancer' -> Kanserli örnekler
# 'Non_Cancer' -> Kanserli olmayan örnekler
labels = ['Cancer', 'Non_Cancer']

# Görüntülerin bulunduğu ana dizin yolunu belirtiyoruz
# Bu klasör içinde 'Cancer' ve 'Non_Cancer' alt klasörleri olabilir
img_path = 'Skin_Data/'

In [4]:
#ls

In [5]:
# Boş bir liste oluşturuyoruz; görüntülerin dosya yollarını buraya ekleyeceğiz
img_list = []

# Boş bir liste oluşturuyoruz; her görüntüye karşılık gelen etiketi buraya ekleyeceğiz
label_list = []

# labels listesindeki her sınıf (ör. 'Cancer' ve 'Non_Cancer') için döngü
for label in labels:
    
    # Her sınıfın klasöründeki tüm dosyaları listele
    for img_file in os.listdir(img_path + label):
        
        # Görüntü dosyasının tam yolunu oluştur ve img_list'e ekle
        img_list.append(img_path + label + "/" + img_file)
        
        # Görüntünün ait olduğu sınıf etiketini label_list'e ekle
        label_list.append(label)

In [6]:
# img_list ve label_list listelerini kullanarak bir DataFrame oluşturuyoruz
# DataFrame, tablo şeklinde veri tutmamızı sağlar (sütunlar: 'img' ve 'label')
df = pd.DataFrame({'img': img_list,  # 'img' sütunu: görüntü dosya yolları
                   'label': label_list})  # 'label' sütunu: her görüntünün sınıf etiketi

In [7]:
df

,img,label
0,Skin_Data/Cancer/1007-1.jpg,Cancer
1,Skin_Data/Cancer/1010-01.JPG,Cancer
2,Skin_Data/Cancer/1012-2.JPG,Cancer
3,Skin_Data/Cancer/1031-1.jpg,Cancer
4,Skin_Data/Cancer/1051-3(94).jpg,Cancer
...,...,...
283,Skin_Data/Non_Cancer/953-1.JPG,Non_Cancer
284,Skin_Data/Non_Cancer/954-3.JPG,Non_Cancer
285,Skin_Data/Non_Cancer/955.JPG,Non_Cancer
286,Skin_Data/Non_Cancer/984.JPG,Non_Cancer


In [8]:
# cancer = 1
# non_cancer = 0 yazdırmak istersek;

In [9]:
# Sınıf isimlerini sayısal değerlere eşleyecek bir sözlük oluşturuyoruz
# 'Cancer' -> 1, 'Non_Cancer' -> 0
d = {'Cancer': 1, 'Non_Cancer': 0}

# DataFrame'deki 'label' sütununu sayısal değerlere çeviriyoruz
# map fonksiyonu, her etiketi sözlükteki karşılığı ile değiştirir
df['label_encoded'] = df['label'].map(d)

In [10]:
df

#Oluşturduğumuz dataframein tamamını görüntüleyebiliriz.

,img,label,label_encoded
0,Skin_Data/Cancer/1007-1.jpg,Cancer,1
1,Skin_Data/Cancer/1010-01.JPG,Cancer,1
2,Skin_Data/Cancer/1012-2.JPG,Cancer,1
3,Skin_Data/Cancer/1031-1.jpg,Cancer,1
4,Skin_Data/Cancer/1051-3(94).jpg,Cancer,1
...,...,...,...
283,Skin_Data/Non_Cancer/953-1.JPG,Non_Cancer,0
284,Skin_Data/Non_Cancer/954-3.JPG,Non_Cancer,0
285,Skin_Data/Non_Cancer/955.JPG,Non_Cancer,0
286,Skin_Data/Non_Cancer/984.JPG,Non_Cancer,0


### _Classification_

In [11]:
# Boş bir liste oluşturuyoruz; işlenmiş görüntüleri buraya ekleyeceğiz
x = []  

# DataFrame'deki tüm görüntü dosya yolları üzerinde döngü
for img in df['img']:
    
    # Görüntüyü OpenCV ile oku
    img = cv2.imread(str(img))
    
    # Görüntüyü 170x170 boyutuna yeniden boyutlandır
    # Not: ResNet transfer learning kullanacağımız için 170x170 girdik
    # Ama başka projelerde yeterli RAM varsa istediğimiz başka boyutu da kullanabiliriz
    img = cv2.resize(img, (170, 170))
    
    # Piksel değerlerini normalize et (0-1 aralığına getir)
    img = img / 255.0
    
    # İşlenmiş görüntüyü listeye ekle
    x.append(img)

In [ ]:
# eğer üstteki kod çalışmazsa aşağıdakini dene

In [ ]:
from tqdm import tqdm

x = []

for img in tqdm(df['img']):

    # Görüntüyü oku
    img = cv2.imread(str(img))

    if img is None:
        print("Bozuk veya okunamayan dosya:", img)
        continue  # bu dosyayı atla

    # 170x170 boyutuna yeniden boyutlandır
    img = cv2.resize(img, (170, 170))

    # Piksel değerlerini normalize et
    img = img / 255.0

    # İşlenmiş görüntüyü listeye ekle
    x.append(img)

In [12]:
# x listesini NumPy dizisine çeviriyoruz
# Çünkü makine öğrenmesi ve derin öğrenme kütüphaneleri genellikle NumPy dizileri ile çalışır
x = np.array(x)  # Artık x, modelin anlayacağı şekilde çok boyutlu bir dizi (array)

In [13]:
x.shape

(288, 170, 170, 3)

In [14]:
# DataFrame'den sadece 'label_encoded' sütununu seçiyoruz
# Bu sütun, her görüntünün sayısal sınıf etiketini (0 veya 1) içeriyor
y = df[['label_encoded']]

In [15]:
# Veriyi eğitim ve test setlerine ayırıyoruz
# x → görüntü verileri (girdi)
# y → etiketler (çıktı)
# test_size=0.20 → verinin %20'si test seti, %80'i eğitim seti olacak
# random_state=42 → rastgele bölme işleminin tekrarlanabilir olmasını sağlar
#                  42 sadece farklı bir sabit sayı, aynı sayıyı her kullanışta aynı bölme elde edilir
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=42)

In [16]:
# Eğitim etiketlerini NumPy dizisine çeviriyoruz ve veri tipini int32 olarak belirliyoruz
y_train = np.array(y_train, dtype=np.int32)

# Test etiketlerini NumPy dizisine çeviriyoruz ve veri tipini int32 olarak belirliyoruz
y_test = np.array(y_test, dtype=np.int32)

In [17]:
# Sequential model oluşturuyoruz, katman katman model ekleyebileceğiz
model = Sequential()

# Modelin giriş katmanını tanımlıyoruz
# Girdi boyutu: 170x170 piksel, 3 renk kanalı (RGB)
model.add(Input(shape=(170, 170, 3)))

# 1. Convolution (evrişim) katmanı
# 32 filtre, 3x3 boyutunda, aktivasyon fonksiyonu ReLU
# Görüntüden özellikler çıkarır
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu'))

# 1. MaxPooling katmanı
# 2x2 boyutunda, uzaysal boyutları küçültür ve hesaplamayı azaltır
model.add(MaxPooling2D(pool_size=(2, 2)))

# 2. Convolution katmanı
# 64 filtre, 3x3 boyutunda, aktivasyon fonksiyonu ReLU
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))

# 2. MaxPooling katmanı
model.add(MaxPooling2D(pool_size=(2, 2)))

# Flatten katmanı
# Çok boyutlu veriyi tek boyuta indirger, Dense katmanına girdi olarak verir
model.add(Flatten())

# Tam bağlantılı (Dense) katman
# 128 nöron, ReLU aktivasyonu
# Görüntüden çıkarılan özellikleri birleştirir ve öğrenir
model.add(Dense(128, activation='relu'))

# Çıkış katmanı
# 1 nöron, sigmoid aktivasyonu
# Binary classification için 0-1 arasında tahmin verir
model.add(Dense(1, activation='sigmoid'))

# Modeli derliyoruz
# optimizer='adam' → ağırlıkları güncellemek için Adam optimizasyonu
# loss='binary_crossentropy' → binary sınıflandırma için uygun kayıp fonksiyonu
# metrics=['accuracy'] → eğitimi izlerken doğruluk metriğini hesaplar
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [18]:
# Modeli eğitim verisiyle eğitiyoruz
history = model.fit(
    x_train,              # Girdi verileri (görüntüler)
    y_train,              # Hedef etiketler (0 veya 1)
    validation_data=(x_test, y_test),  # Her epoch sonunda test verisi ile doğrulama
    epochs=20,            # Modelin tüm eğitim verisi üzerinden 20 kez geçmesi
    verbose=1             # Eğitim sırasında ilerleme çubuğunu göster
)

Epoch 1/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 4s 263ms/step - accuracy: 0.6174 - loss: 1.3681 - val_accuracy: 0.7414 - val_loss: 0.7490
Epoch 2/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 2s 217ms/step - accuracy: 0.7043 - loss: 0.6728 - val_accuracy: 0.7759 - val_loss: 0.6040
Epoch 3/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 2s 222ms/step - accuracy: 0.7174 - loss: 0.5826 - val_accuracy: 0.7931 - val_loss: 0.5361
Epoch 4/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 2s 219ms/step - accuracy: 0.8000 - loss: 0.4828 - val_accuracy: 0.7414 - val_loss: 0.6805
Epoch 5/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 2s 218ms/step - accuracy: 0.7783 - loss: 0.4863 - val_accuracy: 0.8276 - val_loss: 0.3771
Epoch 6/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 2s 220ms/step - accuracy: 0.8087 - loss: 0.4160 - val_accuracy: 0.8621 - val_loss: 0.3735
Epoch 7/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 2s 221ms/step - accuracy: 0.8435 - loss: 0.3326 - val_accuracy: 0.8103 - val_loss: 0.4026
Epoch 8/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 2s 223ms/step - accuracy: 0.8696 - loss: 0.2885 - val_accuracy: 0.8621 - val_loss:

### _Save the Model_

In [19]:
model.save('skin_cancer_model.keras') # huggine kaydetmek istiyorsam .h5 ile kaydetmem lazım

### _Transfer Learning_

_Akıllı insan aklını kullanandır. Daha da akıllı insan başkalarınında aklını kullanandır_

#### _İmport_

In [20]:
# Keras'ın hazır transfer learning modellerini import ediyoruz
# VGG16 ve ResNet50, önceden ImageNet veri setinde eğitilmiş derin CNN modelleridir
from tensorflow.keras.applications import VGG16, ResNet50

# Görüntü verilerini işlemek ve artırmak (augmentation) için ImageDataGenerator sınıfını import ediyoruz
# Örneğin: döndürme, yakınlaştırma, kaydırma gibi işlemlerle eğitim verisini zenginleştirmek için kullanılır
from tensorflow.keras.preprocessing.image import ImageDataGenerator

#### _İmport Another Model - Classification_

In [21]:
# Ana veri klasörünün yolunu belirliyoruz
data_dir = 'Skin_Data'

# Modelin girdi boyutunu belirliyoruz (224x224 piksel)
img_width, img_heigth = 224, 224

# Eğitim verisi için ImageDataGenerator oluşturuyoruz
# rescale=1/255 → piksel değerlerini 0-1 aralığına normalize ediyor
# validation_split=0.20 → verinin %20'sini doğrulama için ayırıyor
train_datagen = ImageDataGenerator(rescale=1/255, validation_split=0.20)

# Eğitim verilerini klasörden okuyup artırma ve hazırlama
train_datagenerator = train_datagen.flow_from_directory(
    directory=data_dir,              # Ana veri klasörü
    target_size=(img_width,img_heigth),  # Görüntüleri 224x224 boyutuna getir
    class_mode='binary',             # Binary sınıflandırma
    subset='training'                # Eğitim verisi olarak ayır
)

# Test/verifikasyon verilerini hazırlıyoruz
test_datagen = ImageDataGenerator(rescale=1/255)

# Doğrulama verilerini klasörden alıyoruz
test_datagenerator = train_datagen.flow_from_directory(
    directory=data_dir,
    target_size=(img_width,img_heigth),
    class_mode='binary',
    subset='validation'              # Doğrulama seti
)

# Önceden eğitilmiş VGG16 modelini yükle
# weights='imagenet' → ImageNet üzerinde önceden eğitilmiş ağırlıklar
# include_top=False → son sınıflandırma katmanını dahil etme (kendi katmanlarımızı ekleyeceğiz)
base_model = VGG16(weights='imagenet', input_shape=(img_width,img_heigth,3), include_top=False)

# Yeni bir Sequential model oluşturuyoruz
model = Sequential()
model.add(base_model)  # Önceden eğitilmiş VGG16 tabanını ekliyoruz

# VGG16 tabanındaki tüm katmanları donduruyoruz, yani eğitim sırasında güncellenmeyecek
for layer in base_model.layers:
    layer.trainable = False

# Kendi üst katmanlarımızı ekliyoruz
model.add(Flatten())            # Çok boyutlu özellik haritasını tek boyuta çevir
model.add(Dense(1024, activation='relu'))  # Fully connected katman, öğrenilen özellikleri birleştir
model.add(Dense(1, activation='sigmoid'))  # Çıkış katmanı, binary sınıflandırma

# Modeli derliyoruz
model.compile(
    optimizer='adam',                 # Ağırlıkları güncellemek için Adam optimizasyonu
    loss='binary_crossentropy',       # Binary sınıflandırma kayıp fonksiyonu
    metrics=['accuracy']              # Eğitim ve doğrulama sırasında doğruluk metriğini takip et
)

# Modeli eğitim verisiyle eğitiyoruz ve doğrulama verisiyle test ediyoruz
# epochs=10 → veri üzerinden 10 kez geçiyoruz
model.fit(
    train_datagenerator,
    epochs=10,
    validation_data=test_datagenerator
)

Found 232 images belonging to 2 classes.
Found 56 images belonging to 2 classes.
Epoch 1/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 27s 3s/step - accuracy: 0.5819 - loss: 4.9801 - val_accuracy: 0.7143 - val_loss: 2.9046
Epoch 2/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 25s 3s/step - accuracy: 0.6897 - loss: 2.1570 - val_accuracy: 0.2857 - val_loss: 1.5291
Epoch 3/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 25s 3s/step - accuracy: 0.7198 - loss: 0.7894 - val_accuracy: 0.8571 - val_loss: 0.3875
Epoch 4/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 24s 3s/step - accuracy: 0.7974 - loss: 0.4272 - val_accuracy: 0.7500 - val_loss: 0.7630
Epoch 5/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 24s 3s/step - accuracy: 0.8621 - loss: 0.3257 - val_accuracy: 0.8393 - val_loss: 0.3579
Epoch 6/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 25s 3s/step - accuracy: 0.8966 - loss: 0.2025 - val_accuracy: 0.8571 - val_loss: 0.4193
Epoch 7/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 25s 3s/step - accuracy: 0.9483 - loss: 0.1546 - val_accuracy: 0.8393 - val_loss: 0.3899
Epoch 8/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 20s 2s/step - a

#### _Save the Transfer Learning Model_

In [22]:
#model.save('skin_cancer.TL.h5') # hugginface de bu uzantıyı kullanabiliyorum; h5
model.save('skin_cancer.TL.keras')